In [1]:
import parsl
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
from parsl.app.app import bash_app,python_app
from parsl import File

In [2]:
config = Config(
    executors=[ThreadPoolExecutor()],
    lazy_errors=True
)
parsl.load(config)

In [12]:
## Define Apps ##
@bash_app
def Combine(inputs=[], outputs=[], stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/Combine.pl ' + ' '.join(inputs) + ' ' + str(outputs[0])

@bash_app
def SingleChannel(inputs=[], outputs=[], channel='1'):
    #return 'perl ./perl/SingleChannel.pl %s %s %s' % (inputs[0], outputs[0], channel)
    #return 'perl ./perl/SingleChannel.pl %s "'.format(inputs[0]) + ' '.join(outputs) + '" "' + ' '.join(channel) + '"'
    return 'perl ./perl/SingleChannel.pl %s %s %s' % (inputs[0], outputs[0], channel)

@bash_app
def Frequency(column, binType, binValue, inputs=[], outputs=[]):
    return 'perl ./perl/Frequency.pl %s %s %s %s %s' % (inputs[0], outputs[0], column, binType, binValue)

In [13]:
## Analysis Parameters ##

# For Combine:
thresholdAll = ['files/6119.2016.0104.1.thresh', 'files/6203.2016.0104.1.thresh']
combineOut = ['outputs/combineOut_Performance']

# For SingleChannel:
singlechannel_channel = '1'
singleChannelOut = 'outputs/singleChannelOut_Performance'

# For Frequency:
freq_col = '5'
freq_binType = '1'
freq_binValue = '2'
freqOut = ['outputs/freqOut_Performance']

In [14]:
## Workflow ##

# 1) Combine() takes the input threshold files and combines
#    them into a single file with name given by `combineOut`
Combine_future = Combine(inputs=thresholdAll, outputs=combineOut)

# 2) SingleChannel() takes the combined data file and divides it into channels
SingleChannel_future = SingleChannel(inputs=Combine_future.outputs, outputs=[singleChannelOut], 
                                     channel=singlechannel_channel)

# 3) We want to apply the Frequency data transformation to (potentially) multiple files output from
#    SingleChannel
#    We define an array to hold the futures of each transformation, then fill it
Freq_futures = []
for i in range(len(SingleChannel_future.outputs)):
    Freq_futures.append(Frequency(column=freq_col, binType=freq_binType, binValue=freq_binValue,
                        inputs=[SingleChannel_future.outputs[i]], outputs=freqOut))

# Now define proper outputs from the futures
Frequency_outputs = [i.outputs[0] for i in Freq_futures]

# Wait for the final result before exiting.
#x = [FreqOut.result() for FreqOut in Frequency_outputs]

for freqOut in Frequency_outputs:
    x = freqOut.result()
    print("Call completed with exit code:", x)

#print("Call to EventSearch completed with exit code:", str(x))

Call completed with exit code: outputs/freqOut_Performance
